In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/user/project/python/rmsKit")
import os
import torch
from lattice import KH, save_npy
os.environ["CUDA_VISIBLE_DEVICES"] = ""
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import rms_torch
import numpy as np
import rms

# u = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
u = np.load("/home/user/project/python/rmsKit/array/torch/KH/3site/smel/"+
        "Jx_1_Jy_1_Jz_1_hx_0_hz_0/e_100_lr_0.001/loss_7.985083/u/0.npy")
device = torch.device("cuda")
p = dict(
    Jx=1,
    Jy=1,
    Jz=1,
    hx=0,
    hz=0,
)
h_list = KH.local("3site", p)
U = np.kron(u, u)

#* don't forget to add minus sign to h
h_list_prime = [-np.array(U @ h @ U.T) for h in h_list[0]]
save_npy(f"/home/user/project/python/rmsKit/array/torch/KH/3site/smel/"+
        "Jx_1_Jy_1_Jz_1_hx_0_hz_0/e_100_lr_0.001/loss_7.985083/H", h_list_prime)

/home/user/miniconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
H_gpu = torch.tensor(H, dtype=torch.float64, device=device)
model = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u).to(device)
loss_smel = rms_torch.SystemMinimumEnergyLoss(H, device=device).to(device)
loss_sel = rms_torch.SystemEnergyLoss(H, device=device).to(device)
compiled_model = torch.compile(model, dynamic = False, fullgraph=True)


## Before optimization

In [23]:
loss_smel.initializer(torch.eye(4096, device=device, dtype = torch.float64))
sel = loss_sel(torch.eye(4096, device=device, dtype = torch.float64)).item()
smel = loss_smel(torch.eye(4096, device=device, dtype = torch.float64)).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.788146799451779
System minimum energy:  8.195037963688478


## SML optimization 

We optimized minimum eigen value

In [25]:
loss_smel.initializer(compiled_model())
sel = loss_sel(compiled_model()).item()
smel = loss_smel(compiled_model()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.690365032623017
System minimum energy:  7.985082781440802


## SEL optimization


In [37]:
u1 = np.load("/home/user/project/python/rmsKit/array/KH/3site/sel/Jx_1_Jy_1_Jz_1_hx_0_hz_0/M_1/u/0.npy")
model2 = rms_torch.UnitaryRieman(H.shape[0], 8, device=device, u0=u1).to(device)
compiled_model2 = torch.compile(model2, dynamic = False, fullgraph=True)

loss_smel.initializer(compiled_model2())
sel = loss_sel(compiled_model2()).item()
smel = loss_smel(compiled_model2()).item()
print("System finite temperature energy: ", sel)
print("System minimum energy: ", smel)

System finite temperature energy:  10.699970951642108
System minimum energy:  8.030657783532432


In [27]:
h1 = np.load("0.npy")
h2 = np.load("1.npy")
h3 = np.load("2.npy")

In [19]:
np.where(np.abs(-0.11785106 - h_list_prime[0]) < 1E-8)

(array([ 1,  3, 25, 27, 41, 43, 49, 51]),
 array([ 3,  1, 27, 25, 43, 41, 51, 49]))

In [20]:
for i in range(5):
    for j in range(5):
        print(i, j, h_list_prime[0][2 + i*8,2 + j*8])

0 0 -4.496925849443831e-06
0 1 -5.137348502907461e-05
0 2 7.848753057178515e-05
0 3 -1.656699728947993e-05
0 4 -1.778307781966555e-05
1 0 -5.137348502915376e-05
1 1 0.16666670986278273
1 2 -3.103655928599431e-05
1 3 -0.11785629850799827
1 4 1.4818667503460935e-05
2 0 7.84875305717614e-05
2 1 -3.103655928601166e-05
2 2 -0.4999997205711204
2 3 -1.686455903025833e-05
2 4 -2.5938780355883226e-05
3 0 -1.656699728945294e-05
3 1 -0.11785629850799832
3 2 -1.686455903025139e-05
3 3 -0.16666596146430332
3 4 1.043730927097233e-05
4 0 -1.7783077819691516e-05
4 1 1.4818667503448792e-05
4 2 -2.593878035588149e-05
4 3 1.043730927091335e-05
4 4 -0.33333079565882834


In [11]:
np.argwhere(np.abs(h_list_prime[0] - 0.43333375) < 1E-4)

array([], shape=(0, 2), dtype=int64)